# Worksheet 07

Name:  Yicun Wu
UID: U60263210

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [10]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.6349588825330574, 6.252892940862441, 4.155284712829086, 5.388165880635526, 4.443819894473208, 5.869001012973493, 4.763086080669089, 4.603268332897948, 3.259856754208278, 5.648334896680342]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [11]:
c2 = np.random.normal( 8 , 1 , 10).tolist()
print(c2)

[9.667321968979335, 8.416397380484055, 6.170954262070578, 10.557557338008836, 8.564256757144406, 7.28518553893374, 8.764953380233317, 7.147047230205504, 8.8926218004438, 8.842894108752482]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [12]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.648334896680342, 3.259856754208278, 8.842894108752482, 8.8926218004438, 4.603268332897948, 4.763086080669089, 5.869001012973493, 7.147047230205504, 4.443819894473208, 5.388165880635526]
[3.6349588825330574, 6.252892940862441, 4.155284712829086]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Number of components: 2 

Mean of component 1

Mean of component 2

Covariance matrix of both components

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [17]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2))/len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.648334896680342, 3.259856754208278, 4.603268332897948, 4.763086080669089, 5.869001012973493, 4.443819894473208, 5.388165880635526]
[8.842894108752482, 8.8926218004438, 7.147047230205504]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 4.853647550362554,  mean_2 = 8.294187713133928
var_1 = 0.6753142028000719,  var_2 = 0.6583777843401207


They are pretty close to each other

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [18]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]*prob_c[0]/prob_x )
    prob_c1_x.append( pdf_i[1]*prob_c[1]/prob_x  )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.648334896680342
probability of observing that point if it came from cluster 0 =  0.29559902281548667
probability of observing that point if it came from cluster 1 =  0.00018855511357544747
point =  3.259856754208278
probability of observing that point if it came from cluster 0 =  0.03646751065508985
probability of observing that point if it came from cluster 1 =  1.2185207231829198e-13
point =  8.842894108752482
probability of observing that point if it came from cluster 0 =  1.5629056894765545e-08
probability of observing that point if it came from cluster 1 =  0.4281600921165103
point =  8.8926218004438
probability of observing that point if it came from cluster 0 =  1.0088866702754277e-08
probability of observing that point if it came from cluster 1 =  0.4008935205808284
point =  4.603268332897948
probability of observing that point if it came from cluster 0 =  0.5515114607293573
probability of observing that point if it came from cluster 1 =  9.07610243621191e-08
point =

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [19]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum(map(lambda x: x[0] * (x[1] - mean[0])**2, zip(prob_c0_x, data))) , sum(map(lambda x: x[0] * (x[1] - mean[0])**2, zip(prob_c1_x, data)))]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7029623681692589,  P(C_2) = 0.29703763183074106
mean_1 = 4.8636590339588635,  mean_2 = 8.30480741281999
var_1 = 4.890252282008565,  var_2 = 37.11836315516674


It is so much diff since K means has so many limitation such as melting pot pts if data too close.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [22]:
# Update P(C_j)
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]

# Update P(X_i | C_j)
prob_x_c = []
for j in range(k):
    prob_x_c_j = [1 / np.sqrt(2*np.pi*var[j]) * np.exp(-1/2*((x-mean[j])/np.sqrt(var[j]))**2) for x in data]
    prob_x_c.append(prob_x_c_j)

# Update P(C_j | X_i)
prob_c_x = [[prob_x_c[j][i] * prob_c[j] / sum([prob_x_c[l][i] * prob_c[l] for l in range(k)]) for j in range(k)] for i in range(len(data))]

# Print results
for i in range(len(data)):
    print(f"P(C_1|X_{i+1}) = {prob_c_x[i][0]}, P(C_2|X_{i+1}) = {prob_c_x[i][1]}")


P(C_1|X_1) = 0.8706799090978479, P(C_2|X_1) = 0.12932009090215207
P(C_1|X_2) = 0.8759615480135208, P(C_2|X_2) = 0.12403845198647921
P(C_1|X_3) = 0.5645900548726468, P(C_2|X_3) = 0.4354099451273532
P(C_1|X_4) = 0.5547420536843801, P(C_2|X_4) = 0.4452579463156198
P(C_1|X_5) = 0.8862009447573287, P(C_2|X_5) = 0.1137990552426714
P(C_1|X_6) = 0.8852195407327282, P(C_2|X_6) = 0.11478045926727166
P(C_1|X_7) = 0.8642985553665211, P(C_2|X_7) = 0.13570144463347888
P(C_1|X_8) = 0.7957348569009488, P(C_2|X_8) = 0.2042651430990512
P(C_1|X_9) = 0.8867196367220757, P(C_2|X_9) = 0.11328036327792428
P(C_1|X_10) = 0.8766779511324888, P(C_2|X_10) = 0.12332204886751119


This gives the updated probability of each data point belonging to each mixture component, which can be used in the E-step of the EM algorithm.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [24]:
hard_assignment = np.argmax(prob_c_x, axis=0)
print(hard_assignment)

[8 3]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.